In [1]:
from pandas import (
    read_csv,
    DataFrame,
    merge,
)
from numpy import (
    std,
    nan,
)

In [2]:
df = read_csv(
    'ratings_fide_november_2020.csv',
    low_memory=False,
    encoding='latin-1',
)

# Data
Check 'descriptions.txt' for details on the fields. This was taken from here: https://ratings.fide.com/download_lists.phtml 
Field names in the XML file do not exactly match descriptions on the ratings page in FIDE site.
We will assume that 'games' mean the number of standard rated games and 'rating' means the rating of standard games. It's a shame FIDE does not keep data in a standard CSV format. Nobody really uses XML anymore and the txt format they provide has formatting issues with spaces/tabs.

In [3]:
df.head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,k,rapid_rating,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag
0,10688862,"A Abdel Maabod, Hoda",EGY,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009.0,w
1,10224084,"A B M Hasibuzzaman, Tapan",BAN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1977.0,NaN
2,10245154,"A B M Jobair, Hossain",BAN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1599.0,0.0,20.0,NaN,NaN,NaN,1998.0,NaN
3,10243054,"A B M Mustakim, Chowdhury",BAN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,NaN
4,25121731,A C J John,IND,M,NaN,NaN,NaN,NaN,1063.0,0.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,1987.0,NaN


# Checking a particular country

In [4]:
df[df['country'] == 'CHN']

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,k,rapid_rating,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag
21,8605360,"A La, Teng Hua",CHN,F,NaN,NaN,NaN,NaN,1915.0,0.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,1993.0,wi
37,8635170,"A, Sihan",CHN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007.0,NaN
9681,8620270,"Abudureheman, Namaiti",CHN,M,NaN,NaN,NaN,NaN,2169.0,0.0,40.0,1994.0,0.0,20.0,NaN,NaN,NaN,1986.0,NaN
20186,8622930,Ai Kebaier Aikelamu,CHN,M,NaN,NaN,NaN,NaN,1843.0,0.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,i
20188,8610010,"Ai, Erkengjiang",CHN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982297,8630119,"Zuo, Shengyuan",CHN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,NaN
982299,8614610,"Zuo, Yifan",CHN,M,CM,NaN,NaN,NaN,2330.0,0.0,20.0,2060.0,0.0,20.0,2055.0,0.0,20.0,2005.0,NaN
982300,8620490,"Zuo, YiMing",CHN,F,NaN,NaN,NaN,NaN,1055.0,0.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,1990.0,wi
982302,8619832,"Zuo, Zhibo",CHN,M,NaN,NaN,NaN,NaN,1679.0,0.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,i


In [5]:
df.shape

(983184, 19)

# Gender count in full dataset

In [6]:
df['sex'].value_counts()

M    837331
F    145853
Name: sex, dtype: int64

# Cleaning

In [7]:
"""
Add a column with total games
"""
columns = [
    'games',
    'rapid_games',
    'blitz_games',
    'rating',
    'blitz_rating',
    'rapid_rating',
    'k',
    'rapid_k',
    'blitz_k',
]
"""
Replace nan values in these columns by 0
"""
for col in columns:
    df[col] = df[col].fillna(0)
"""
Replace nan values in titles with ''
"""
columns_title = [
    'title',
    'w_title',
    'o_title',
    'foa_title',
]
for col in columns_title:
    df[col] = df[col].fillna(' ')
"""
For flags, change nan to 'a' to denote active
"""
df['flag'] = df['flag'].fillna('a')

# Generating average ratings

In [8]:
df['total_games'] = df['games']+df['rapid_games']+df['blitz_games']
df['average_rating'] = (df['rating']+df['rapid_rating']+df['blitz_rating'])/3.0
df['average_games'] = df['total_games']/3.0

In [9]:
df.head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,...,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag,total_games,average_rating,average_games
0,10688862,"A Abdel Maabod, Hoda",EGY,F,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2009.0,w,0.0,0.000000,0.0
1,10224084,"A B M Hasibuzzaman, Tapan",BAN,M,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1977.0,a,0.0,0.000000,0.0
2,10245154,"A B M Jobair, Hossain",BAN,M,,,,,0.0,0.0,...,0.0,20.0,0.0,0.0,0.0,1998.0,a,0.0,533.000000,0.0
3,10243054,"A B M Mustakim, Chowdhury",BAN,M,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2013.0,a,0.0,0.000000,0.0
4,25121731,A C J John,IND,M,,,,,1063.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1987.0,a,0.0,354.333333,0.0


# Drop non-rated players

In [10]:
df = df.dropna(
    subset=[
        'rating',
    ],
)
df.shape

(983184, 22)

In [11]:
df.head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,...,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag,total_games,average_rating,average_games
0,10688862,"A Abdel Maabod, Hoda",EGY,F,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2009.0,w,0.0,0.000000,0.0
1,10224084,"A B M Hasibuzzaman, Tapan",BAN,M,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1977.0,a,0.0,0.000000,0.0
2,10245154,"A B M Jobair, Hossain",BAN,M,,,,,0.0,0.0,...,0.0,20.0,0.0,0.0,0.0,1998.0,a,0.0,533.000000,0.0
3,10243054,"A B M Mustakim, Chowdhury",BAN,M,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2013.0,a,0.0,0.000000,0.0
4,25121731,A C J John,IND,M,,,,,1063.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1987.0,a,0.0,354.333333,0.0


# Gender count of rated players only

In [12]:
df['sex'].value_counts()

M    837331
F    145853
Name: sex, dtype: int64

# Considering active players only
Drop players who are inactive. Also, why are there two different flags for inactivity based on gender, FIDE? What would be wrong with just one flag that says if the player is active or not?

In [13]:
'''
For now we ignore this
'''
#df = df[~df['flag'].isin(['i','wi'])]
#df = df.reset_index(drop=True)
#df.shape

'\nFor now we ignore this\n'

In [14]:
df.head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,...,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag,total_games,average_rating,average_games
0,10688862,"A Abdel Maabod, Hoda",EGY,F,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2009.0,w,0.0,0.000000,0.0
1,10224084,"A B M Hasibuzzaman, Tapan",BAN,M,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1977.0,a,0.0,0.000000,0.0
2,10245154,"A B M Jobair, Hossain",BAN,M,,,,,0.0,0.0,...,0.0,20.0,0.0,0.0,0.0,1998.0,a,0.0,533.000000,0.0
3,10243054,"A B M Mustakim, Chowdhury",BAN,M,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2013.0,a,0.0,0.000000,0.0
4,25121731,A C J John,IND,M,,,,,1063.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1987.0,a,0.0,354.333333,0.0


# Gender count of active rated players

In [15]:
df['sex'].value_counts()

M    837331
F    145853
Name: sex, dtype: int64

# Sort by country

In [16]:
df = df.sort_values(
    by=[
        'country',
    ],
)
df = df.reset_index(drop=True)

In [17]:
df.head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,...,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag,total_games,average_rating,average_games
0,11702346,"Abdul, Qadeer",AFG,M,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1983.0,a,0.0,0.000000,0.0
1,11702141,"Ahmadzai, Zuhal",AFG,F,,,,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2000.0,w,0.0,0.000000,0.0
2,11701960,"Zhakfar, Abdul Saber",AFG,M,,,,,1427.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1979.0,a,0.0,475.666667,0.0
3,11700432,Zaman,AFG,M,,,IA,,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,a,0.0,0.000000,0.0
4,11700033,"Ahmadi, Zabiullah",AFG,M,,,,,1987.0,0.0,...,0.0,0.0,2024.0,0.0,20.0,1988.0,i,0.0,1337.000000,0.0


# Players with at least one rated game

In [18]:
df = df[df['total_games'] > 0]
df = df.reset_index(drop=True)
df.shape

(21989, 22)

In [19]:
df.head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,...,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag,total_games,average_rating,average_games
0,11701552,"Mohtaat, Homayoun",AFG,M,,,,AIM,1791.0,8.0,...,0.0,20.0,1796.0,0.0,20.0,1972.0,a,8.0,1755.666667,2.666667
1,11701498,Samsoor M Shoaib,AFG,M,,,,,1201.0,0.0,...,4.0,20.0,0.0,0.0,0.0,1989.0,i,4.0,813.666667,1.333333
2,4704673,"Boci, Petro",ALB,M,,,,,1442.0,5.0,...,0.0,0.0,0.0,0.0,0.0,2004.0,a,5.0,480.666667,1.666667
3,4700708,"Pasku, Roela",ALB,F,WFM,WFM,,,1907.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1990.0,w,2.0,635.666667,0.666667
4,4701275,"Gjonaj, Jurgen",ALB,M,,,,,1897.0,1.0,...,0.0,20.0,1869.0,4.0,20.0,1996.0,a,5.0,1863.666667,1.666667


# Gender count of active players with at least one rated game

In [20]:
df['sex'].value_counts()

M    19874
F     2115
Name: sex, dtype: int64

# Separate data based on gender

In [21]:
genders = {
    'woman': 'F',
    'man': 'M',
}
dfs = {}
for gender in genders:
    dfs[gender] = df[df['sex'] == genders[gender]]
    dfs[gender] = dfs[gender].reset_index(drop=True)
    print(gender, dfs[gender].shape[0])

woman 2115
man 19874


# Check data after separation

In [22]:
dfs['man'].head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,...,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag,total_games,average_rating,average_games
0,11701552,"Mohtaat, Homayoun",AFG,M,,,,AIM,1791.0,8.0,...,0.0,20.0,1796.0,0.0,20.0,1972.0,a,8.0,1755.666667,2.666667
1,11701498,Samsoor M Shoaib,AFG,M,,,,,1201.0,0.0,...,4.0,20.0,0.0,0.0,0.0,1989.0,i,4.0,813.666667,1.333333
2,4704673,"Boci, Petro",ALB,M,,,,,1442.0,5.0,...,0.0,0.0,0.0,0.0,0.0,2004.0,a,5.0,480.666667,1.666667
3,4701275,"Gjonaj, Jurgen",ALB,M,,,,,1897.0,1.0,...,0.0,20.0,1869.0,4.0,20.0,1996.0,a,5.0,1863.666667,1.666667
4,4705149,"Kola, Elvis",ALB,M,,,,,1977.0,7.0,...,0.0,20.0,0.0,0.0,0.0,1984.0,a,7.0,1346.333333,2.333333


In [23]:
dfs['woman'].head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,...,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag,total_games,average_rating,average_games
0,4700708,"Pasku, Roela",ALB,F,WFM,WFM,,,1907.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1990.0,w,2.0,635.666667,0.666667
1,4705807,"Ndreko, Ailin",ALB,F,,,,,0.0,0.0,...,5.0,20.0,0.0,0.0,0.0,2012.0,w,5.0,354.333333,1.666667
2,4701984,"Shqau, Olta",ALB,F,,,,,1340.0,4.0,...,0.0,0.0,0.0,0.0,0.0,2002.0,w,4.0,446.666667,1.333333
3,4701585,"Shuqja, Klean",ALB,F,WCM,WCM,,,1905.0,12.0,...,0.0,20.0,1760.0,0.0,20.0,2006.0,w,12.0,1743.666667,4.000000
4,7900414,"Nouali, Djouher",ALG,F,,,,,1748.0,1.0,...,0.0,20.0,1728.0,0.0,20.0,1987.0,w,1.0,1713.333333,0.333333


# Basic stats of woman ratings

In [24]:
dfs['woman']['rating'].describe()

count    2115.000000
mean     1473.949882
std       546.073327
min         0.000000
25%      1218.000000
50%      1523.000000
75%      1852.000000
max      2592.000000
Name: rating, dtype: float64

# Basic stats of man ratings

In [25]:
dfs['man']['rating'].describe()

count    19874.000000
mean      1674.569437
std        529.740856
min          0.000000
25%       1467.000000
50%       1766.000000
75%       2007.000000
max       2862.000000
Name: rating, dtype: float64

# Aggregate ratings by country for each gender

In [26]:
for gender in genders:
    column = gender+'_country'
    dfs[column] = dfs[gender].groupby(
        by=[
            'country',
        ],
        as_index=False,
    )[['rating', 'title']].agg(lambda x: list(x))
    dfs[column]['rating'] = dfs[column]['rating'].apply(lambda x: sorted(x))
    dfs[column]['count'] = dfs[column]['rating'].apply(lambda x: len(x))
    dfs[column]['total'] = dfs[column]['rating'].apply(lambda x: sum(x))
    dfs[column]['average'] = dfs[column]['total']/dfs[column]['count']
    dfs[column]['std'] = dfs[column]['rating'].apply(lambda x: std(x))
    dfs[column] = round(dfs[column], 2)
    dfs[column] = dfs[column].sort_values(
        by=[
            'count',
            'average',
            'country',
        ],
    )
    dfs[column] = dfs[column].reset_index(drop=True)
    print(gender, dfs[column].shape[0])

woman 70
man 109


In [27]:
dfs['man_country'].head()

,country,rating,title,count,total,average,std
0,CHN,[0.0],[ ],1,0.0,0.0,0.0
1,HKG,[0.0],[ ],1,0.0,0.0,0.0
2,TPE,[1110.0],[ ],1,1110.0,1110.0,0.0
3,JPN,[1201.0],[ ],1,1201.0,1201.0,0.0
4,SCO,[1613.0],[ ],1,1613.0,1613.0,0.0


In [28]:
dfs['woman_country'].head()

,country,rating,title,count,total,average,std
0,USA,[1105.0],[ ],1,1105.0,1105.0,0.0
1,SWZ,[1368.0],[ ],1,1368.0,1368.0,0.0
2,TUN,[1463.0],[ ],1,1463.0,1463.0,0.0
3,SYR,[1581.0],[ ],1,1581.0,1581.0,0.0
4,NED,[1605.0],[ ],1,1605.0,1605.0,0.0


# Join these two by country

In [29]:
dfs['combined_country'] = merge(
    left=dfs['man_country'],
    right=dfs['woman_country'],
    left_on=[
        'country',
    ],
    right_on=[
        'country',
    ],
    how='outer',
)
dfs['combined_country'] = dfs['combined_country'].sort_values(
    by=[
        'country',
    ],
)
dfs['combined_country'] = dfs['combined_country'].reset_index(drop=True)
cols = [
    'count_x',
    'count_y',
    'total_x',
    'total_y',
    'average_x',
    'average_y',
]
for col in cols:
    dfs['combined_country'][col] = dfs['combined_country'][col].fillna(0)

In [30]:
dfs['combined_country'].shape

(111, 13)

In [31]:
dfs['combined_country'].head()

,country,rating_x,title_x,count_x,total_x,average_x,std_x,rating_y,title_y,count_y,total_y,average_y,std_y
0,AFG,"[1201.0, 1791.0]","[ , ]",2.0,2992.0,1496.0,295.00,NaN,NaN,0.0,0.0,0.0,NaN
1,ALB,"[1442.0, 1482.0, 1557.0, 1563.0, 1616.0, 1658....","[ , , , IM, , , , , , , IM, , , , ...",17.0,32215.0,1895.0,317.43,"[0.0, 1340.0, 1905.0, 1907.0]","[WFM, , , WCM]",4.0,5152.0,1288.0,778.7
2,ALG,"[1789.0, 1939.0, 1980.0, 2005.0, 2012.0, 2054....","[ , GM, , FM, , , , , , CM]",10.0,20574.0,2057.4,176.63,"[1734.0, 1748.0]","[ , WFM]",2.0,3482.0,1741.0,7.0
3,AND,"[1926.0, 2072.0]","[ , ]",2.0,3998.0,1999.0,73.00,NaN,NaN,0.0,0.0,0.0,NaN
4,ANG,[1942.0],[ ],1.0,1942.0,1942.0,0.00,NaN,NaN,0.0,0.0,0.0,NaN


In [32]:
dfs['combined_country']['average_diff'] = dfs['combined_country']['average_x']-dfs['combined_country']['average_y']
dfs['combined_country']['count_diff'] = dfs['combined_country']['count_x']-dfs['combined_country']['count_y']

In [33]:
dfs['combined_country'].head()

,country,rating_x,title_x,count_x,total_x,average_x,std_x,rating_y,title_y,count_y,total_y,average_y,std_y,average_diff,count_diff
0,AFG,"[1201.0, 1791.0]","[ , ]",2.0,2992.0,1496.0,295.00,NaN,NaN,0.0,0.0,0.0,NaN,1496.0,2.0
1,ALB,"[1442.0, 1482.0, 1557.0, 1563.0, 1616.0, 1658....","[ , , , IM, , , , , , , IM, , , , ...",17.0,32215.0,1895.0,317.43,"[0.0, 1340.0, 1905.0, 1907.0]","[WFM, , , WCM]",4.0,5152.0,1288.0,778.7,607.0,13.0
2,ALG,"[1789.0, 1939.0, 1980.0, 2005.0, 2012.0, 2054....","[ , GM, , FM, , , , , , CM]",10.0,20574.0,2057.4,176.63,"[1734.0, 1748.0]","[ , WFM]",2.0,3482.0,1741.0,7.0,316.4,8.0
3,AND,"[1926.0, 2072.0]","[ , ]",2.0,3998.0,1999.0,73.00,NaN,NaN,0.0,0.0,0.0,NaN,1999.0,2.0
4,ANG,[1942.0],[ ],1.0,1942.0,1942.0,0.00,NaN,NaN,0.0,0.0,0.0,NaN,1942.0,1.0


In [34]:
dfs['combined_country'][dfs['combined_country']['count_diff'] < 0].shape[0]

2

In [35]:
dfs['combined_country']['count_diff'].describe()

count     111.000000
mean      159.990991
std       479.736585
min        -1.000000
25%         3.000000
50%        13.000000
75%        93.500000
max      4515.000000
Name: count_diff, dtype: float64

In [36]:
dfs['combined_country']['average_diff'].describe()

count     111.000000
mean      743.946126
std       939.021091
min     -1877.000000
25%        94.325000
50%       273.460000
75%      1758.325000
max      2504.200000
Name: average_diff, dtype: float64